<a href="https://colab.research.google.com/github/OneFineStarstuff/State-of-the-Art/blob/main/GCN_Implementation_using_PyTorch_Geometric_(Define_Everything_Inline).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch_geometric

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data

# --- GCN Definition ---
class GCN(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def forward(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))  # First GCN layer + ReLU
        x = self.conv2(x, edge_index)          # Output layer (raw logits)
        return x

# --- Graph Example: 3 nodes, edges, and features ---
x = torch.randn(3, 5)  # Node features: [num_nodes, num_features]

# Edges: edge_index has shape [2, num_edges]
# Format: [source_nodes; target_nodes]
edge_index = torch.tensor([
    [0, 1, 1, 2],  # from nodes
    [1, 0, 2, 1]   # to nodes
], dtype=torch.long)

# Wrap graph data
data = Data(x=x, edge_index=edge_index)

# --- Instantiate & Run Model ---
model = GCN(in_channels=5, hidden_channels=16, out_channels=2)
output = model(data.x, data.edge_index)

print("GCN Output Shape:", output.shape)
print("Node-level Output:\n", output)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data

# --- Define GCN Model ---
class GCN(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def forward(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        x = self.conv2(x, edge_index)
        return x

# --- Synthetic Graph Data ---
x = torch.randn(6, 5)  # 6 nodes, each with 5 features

edge_index = torch.tensor([
    [0, 1, 2, 3, 4, 5, 1, 2],
    [1, 0, 3, 2, 5, 4, 2, 1]
], dtype=torch.long)

y = torch.tensor([0, 1, 0, 1, 0, 1], dtype=torch.long)  # Labels for 6 nodes

# Define train and test masks
train_mask = torch.tensor([True, True, True, False, False, False])
test_mask = ~train_mask  # Inverse mask for testing

data = Data(x=x, edge_index=edge_index, y=y, train_mask=train_mask, test_mask=test_mask)

# --- Training Setup ---
model = GCN(in_channels=5, hidden_channels=8, out_channels=2)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

# --- Training Loop ---
for epoch in range(1, 201):
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    loss = criterion(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

    if epoch % 20 == 0 or epoch == 1:
        model.eval()
        _, pred = out.max(dim=1)
        correct = int((pred[data.test_mask] == data.y[data.test_mask]).sum())
        acc = correct / int(data.test_mask.sum())
        print(f"Epoch {epoch:3d} | Loss: {loss.item():.4f} | Test Accuracy: {acc:.2f}")

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv

# --- Load Cora Dataset ---
dataset = Planetoid(root='./data', name='Cora')
data = dataset[0]

# --- Define GCN Model ---
class GCN(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def forward(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        x = self.conv2(x, edge_index)
        return x

# --- Initialize Model, Loss, Optimizer ---
model = GCN(dataset.num_node_features, 16, dataset.num_classes)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
criterion = nn.CrossEntropyLoss()

# --- Training Loop ---
model.train()
for epoch in range(1, 201):
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    loss = criterion(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

    if epoch % 20 == 0 or epoch == 1:
        model.eval()
        _, pred = out.max(dim=1)
        correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()
        acc = int(correct) / int(data.test_mask.sum())
        print(f"Epoch {epoch:3d} | Loss: {loss.item():.4f} | Test Accuracy: {acc:.2f}")
        model.train()

In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

model.eval()
with torch.no_grad():
    z = model(data.x, data.edge_index)

z = z.cpu().numpy()
labels = data.y.cpu().numpy()

tsne = TSNE(n_components=2)
z_2d = tsne.fit_transform(z)

plt.figure(figsize=(8, 6))
plt.scatter(z_2d[:, 0], z_2d[:, 1], c=labels, cmap='jet', s=15)
plt.title("t-SNE of GCN Node Embeddings")
plt.show()

In [ ]:
from torch_geometric.nn import GATConv  # or GINConv, SAGEConv

In [ ]:
import networkx as nx
from torch_geometric.utils import from_networkx

G = nx.karate_club_graph()
data = from_networkx(G)

In [ ]:
class GCN(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.bn1 = nn.BatchNorm1d(hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = self.bn1(x)
        x = F.relu(x)
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index)
        return x

In [ ]:
from torch_geometric.nn import GATConv

class GAT(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.gat1 = GATConv(in_channels, hidden_channels, heads=8, dropout=0.6)
        self.gat2 = GATConv(hidden_channels * 8, out_channels, heads=1, concat=False, dropout=0.6)

    def forward(self, x, edge_index):
        x = self.gat1(x, edge_index)
        x = F.elu(x)
        x = self.gat2(x, edge_index)
        return x

In [ ]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

# inside your loop
writer.add_scalar("Loss/train", loss.item(), epoch)
writer.add_scalar("Accuracy/test", acc, epoch)

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, GATConv

class GCN(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def forward(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, p=0.5, training=self.training)
        return self.conv2(x, edge_index)

class GAT(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.gat1 = GATConv(in_channels, hidden_channels, heads=8, dropout=0.6)
        self.gat2 = GATConv(hidden_channels * 8, out_channels, heads=1, concat=False, dropout=0.6)

    def forward(self, x, edge_index):
        x = F.elu(self.gat1(x, edge_index))
        x = F.dropout(x, p=0.6, training=self.training)
        return self.gat2(x, edge_index)

In [ ]:
from torch_geometric.datasets import Planetoid

def load_cora():
    dataset = Planetoid(root='./data', name='Cora')
    return dataset[0], dataset.num_node_features, dataset.num_classes

In [ ]:
import torch
import torch.nn.functional as F

def train(model, data, optimizer, criterion, epochs=200):
    for epoch in range(1, epochs + 1):
        model.train()
        optimizer.zero_grad()
        out = model(data.x, data.edge_index)
        loss = criterion(out[data.train_mask], data.y[data.train_mask])
        loss.backward()
        optimizer.step()

        if epoch % 20 == 0 or epoch == 1:
            acc = evaluate(model, data)
            print(f"Epoch {epoch} | Loss: {loss.item():.4f} | Test Acc: {acc:.2f}")

def evaluate(model, data):
    model.eval()
    with torch.no_grad():
        logits = model(data.x, data.edge_index)
        pred = logits.argmax(dim=1)
        correct = pred[data.test_mask] == data.y[data.test_mask]
        return int(correct.sum()) / int(data.test_mask.sum())

In [ ]:
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import torch

def visualize(model, data):
    model.eval()
    with torch.no_grad():
        z = model(data.x, data.edge_index).cpu().numpy()
        labels = data.y.cpu().numpy()

    z_embedded = TSNE(n_components=2).fit_transform(z)
    plt.scatter(z_embedded[:, 0], z_embedded[:, 1], c=labels, cmap='tab10', s=15)
    plt.title("Node Embeddings (t-SNE)")
    plt.show()

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from model import GCN  # or GAT
from dataset import load_cora
from train import train
from visualize import visualize

# Load dataset
data, in_channels, out_classes = load_cora()

# Initialize model
model = GCN(in_channels, 16, out_classes)
optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
criterion = nn.CrossEntropyLoss()

# Train and visualize
train(model, data, optimizer, criterion)
visualize(model, data)

In [ ]:
from torch_geometric.nn import GCNConv
import torch.nn.functional as F
import torch.nn as nn

class GCN(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def forward(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        x = self.conv2(x, edge_index)
        return x

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

# GCN Model
class GCN(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def forward(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, p=0.5, training=self.training)
        return self.conv2(x, edge_index)

# Load dataset
dataset = Planetoid(root="./data", name="Cora")
data = dataset[0]

# Model, optimizer, loss
model = GCN(dataset.num_node_features, 16, dataset.num_classes)
optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
criterion = nn.CrossEntropyLoss()

# Training loop
for epoch in range(1, 201):
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    loss = criterion(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

    if epoch % 20 == 0 or epoch == 1:
        model.eval()
        _, pred = out.max(dim=1)
        correct = pred[data.test_mask] == data.y[data.test_mask]
        acc = int(correct.sum()) / int(data.test_mask.sum())
        print(f"Epoch {epoch:3d} | Loss: {loss.item():.4f} | Test Acc: {acc:.2f}")

# t-SNE visualization
model.eval()
with torch.no_grad():
    z = model(data.x, data.edge_index).cpu().numpy()
    labels = data.y.cpu().numpy()

z_emb = TSNE(n_components=2).fit_transform(z)
plt.figure(figsize=(8, 6))
plt.scatter(z_emb[:, 0], z_emb[:, 1], c=labels, cmap="tab10", s=15)
plt.title("t-SNE of GCN Node Embeddings on Cora")
plt.show()